<a href="https://colab.research.google.com/github/ps08022009/nlp-with-bert-model/blob/main/NLP_Bert_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [2]:
!pip install torch torchvision torchaudio

In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

#Instantiate Model

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

#Calculate Sentiment

In [13]:
response=input("Was the food to your liking? ")

Was the food to your liking? no


In [14]:
tokens = tokenizer.encode(response, return_tensors='pt')

In [15]:
result = model(tokens)

In [16]:
result.logits

tensor([[ 1.9555,  1.3763,  0.6166, -1.3143, -1.9839]],
       grad_fn=<AddmmBackward0>)

In [17]:
int(torch.argmax(result.logits))+1

1

#Collect Reviews


In [18]:
site = input("What website would you like to scrape? ")
r = requests.get(site)
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

What website would you like to scrape? https://www.yelp.com/biz/the-landing-renton


In [19]:
reviews

["It's been 8 years since I reviewed the Landing and that review is very outdated!When I reviewed The Landing last It was still largely a ghost town. The large anchor spaces were filled like Ross, LA Fitness, Red Robin, and The Rock But most of the smaller spaces were vacant. As the years passed and the economy improved more stores and restaurants occupied The Landing to the point I'd wager it is mostly full. They offer some services like medical, dental, chiropractor, and vision while also having a large selection of restaurants. Some of these restaurants are the quick bite variety such as Potbelly Sandwiches and Mucho Burrito while others offer a longer dining experience like Wild Fin or Trenchers. By and large they have more chain restaurants and stores than smaller businesses, and sadly the smaller businesses seem to have the highest turnover. I still miss Pongi Bonsi and the cute chocolate shop. I hear rent there can be very steep. If you want to go to Ross, Target, Marshall's, or

#Data Frame

In [24]:
import numpy as np
import pandas as pd

In [25]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [26]:
df['review'].iloc[0]

"It's been 8 years since I reviewed the Landing and that review is very outdated!When I reviewed The Landing last It was still largely a ghost town. The large anchor spaces were filled like Ross, LA Fitness, Red Robin, and The Rock But most of the smaller spaces were vacant. As the years passed and the economy improved more stores and restaurants occupied The Landing to the point I'd wager it is mostly full. They offer some services like medical, dental, chiropractor, and vision while also having a large selection of restaurants. Some of these restaurants are the quick bite variety such as Potbelly Sandwiches and Mucho Burrito while others offer a longer dining experience like Wild Fin or Trenchers. By and large they have more chain restaurants and stores than smaller businesses, and sadly the smaller businesses seem to have the highest turnover. I still miss Pongi Bonsi and the cute chocolate shop. I hear rent there can be very steep. If you want to go to Ross, Target, Marshall's, or 

In [27]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [28]:
sentiment_score(df['review'].iloc[1])

4

In [29]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [32]:
df

,review,sentiment
0,It's been 8 years since I reviewed the Landing...,4
1,The Landing as a whole is a pretty part of Ren...,4
2,Most of the businesses close by 6 pm on a Sund...,3
3,Watch out they tow within minutes if you park ...,1
4,"Clean, Pleasant, Convenient Urban Village styl...",4
5,The Landing has been a great addition to Rento...,4
6,One of my go to places when I'm in town. Great...,5
7,Absolutely nothing there! Just a few shops and...,2
8,Very decent location just off I-405 freeway! I...,4
9,Love this place. So much shops to go shopping....,5


import numpy as np
from google.colab import autoviz
df_3631793523787470058 = autoviz.get_df('df_3631793523787470058')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_3631793523787470058, *['sentiment'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3631793523787470058 = autoviz.get_df('df_3631793523787470058')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_3631793523787470058, *['sentiment'], **{})
chart

In [31]:
df['review'].iloc[3]

'Watch out they tow within minutes if you park there and have to go across the street. \xa0Never shopping there again. \xa0700 tow bill is bent over pleasure, seems illegal.'